In [25]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [49]:
# config variables
ROOT = "C:/Users/Beau/Documents/GitHub/RealEstate"
SERIALIZED = os.path.join(ROOT, "data", "serialized")
CLEANED = os.path.join(ROOT, "data", "processed")

In [50]:
df_input = pd.read_pickle(os.path.join(CLEANED, "input.pkl"))

In [51]:
df_input.columns.tolist()

['index',
 'PROPERTY TYPE',
 'ADDRESS',
 'ZIP OR POSTAL CODE',
 'PRICE',
 'BEDS',
 'BATHS',
 'LOCATION',
 'SQUARE FEET',
 'LOT SIZE',
 'YEAR BUILT',
 'DAYS ON MARKET',
 '$/SQUARE FEET',
 'HOA/MONTH',
 'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
 'MLS#',
 'LATITUDE',
 'LONGITUDE',
 'min_dist_to_grocery',
 'min_dist_to_mural',
 'min_dist_to_art',
 'count_grocery_0.5_mi',
 'count_murals_0.5_mi',
 'count_art_0.5_mi']

In [52]:
# handle missing data
print(df_input.isnull().sum())

# remove listing without number of bathrooms
df_input = df_input.loc[df_input["BATHS"].notnull()]

# if HOA fees missing, assume there are none
df_input.loc[df_input["HOA/MONTH"].isnull(), "HOA/MONTH"] = 0

# if year built is missing, use the average from the other data
avg_year_built = df_input["YEAR BUILT"].mean()
df_input.loc[df_input["YEAR BUILT"].isnull(), "YEAR BUILT"] = avg_year_built

df_input.reset_index(drop=True)

index                                                                                            0
PROPERTY TYPE                                                                                    0
ADDRESS                                                                                          0
ZIP OR POSTAL CODE                                                                               0
PRICE                                                                                            0
BEDS                                                                                             0
BATHS                                                                                            1
LOCATION                                                                                         0
SQUARE FEET                                                                                     92
LOT SIZE                                                                                       203
YEAR BUILT

,index,PROPERTY TYPE,ADDRESS,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,...,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),MLS#,LATITUDE,LONGITUDE,min_dist_to_grocery,min_dist_to_mural,min_dist_to_art,count_grocery_0.5_mi,count_murals_0.5_mi,count_art_0.5_mi
0,0,Condo/Co-op,3001 W Logan Blvd #1,60647,312900,2.0,1.0,CHI - Logan Square,950.0,NaN,...,https://www.redfin.com/IL/Chicago/3001-W-Logan...,11689023,41.927859,-87.702645,0.459995,0.324022,0.675326,1,3,0
1,1,Condo/Co-op,317 E 25th Pl Unit 3W,60616,290000,2.0,2.0,CHI - Near South Side,1214.0,NaN,...,https://www.redfin.com/IL/Chicago/317-E-25th-S...,11687898,41.847339,-87.619935,0.718396,0.730189,0.573372,0,0,0
2,2,Multi-Family (2-4 Unit),7847 S Saint Lawrence Ave,60619,225000,4.0,2.0,CHI - Greater Grand Crossing,NaN,NaN,...,https://www.redfin.com/IL/Chicago/7847-S-St-La...,11688895,41.751655,-87.609821,0.447490,1.123216,2.167356,1,0,0
3,3,Condo/Co-op,933 W Van Buren St #324,60607,290000,1.0,1.0,CHI - Near West Side,800.0,NaN,...,https://www.redfin.com/IL/Chicago/933-W-Van-Bu...,11686799,41.876095,-87.651260,0.223117,0.842772,0.449744,4,0,2
4,4,Single Family Residential,19 E 102nd Pl,60628,130000,3.0,2.0,CHI - Roseland,1633.0,4687.0,...,https://www.redfin.com/IL/Chicago/19-E-102nd-P...,11688879,41.707893,-87.622745,0.444903,1.218160,2.617911,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,344,Condo/Co-op,6025 S Saint Lawrence Ave #2,60637,179000,3.0,2.0,CHI - Woodlawn,NaN,NaN,...,https://www.redfin.com/IL/Chicago/6025-S-St-La...,11688522,41.785037,-87.610586,0.237449,0.371764,0.189429,3,1,1
305,345,Single Family Residential,5831 W Newport Ave,60634,599000,5.0,3.5,CHI - Portage Park,2668.0,3781.0,...,https://www.redfin.com/IL/Chicago/5831-W-Newpo...,11688523,41.942848,-87.772784,0.386474,0.961685,0.862762,1,0,0
306,346,Condo/Co-op,4158 W Cullom Ave,60641,795000,3.0,3.5,CHI - Irving Park,NaN,NaN,...,https://www.redfin.com/IL/Chicago/4158-W-Cullo...,11688517,41.959507,-87.731724,0.558228,0.556612,1.101335,0,0,0
307,347,Townhouse,1211 W Lexington St,60607,600000,3.0,3.0,Columbus on the Park,2400.0,NaN,...,https://www.redfin.com/IL/Chicago/1211-W-Lexin...,11688514,41.872068,-87.657340,0.419165,0.741022,0.208722,1,0,2


In [83]:
X = df_input[[
    "PROPERTY TYPE",
#     "ZIP OR POSTAL CODE",
    "BEDS",
    "BATHS",
#     "LOCATION",
#     "SQUARE FEET",
#     "LOT SIZE",
    "YEAR BUILT",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "min_dist_to_grocery",
    "min_dist_to_mural",
    "min_dist_to_art",
    "count_grocery_0.5_mi",
    "count_murals_0.5_mi",
    "count_art_0.5_mi"
]].reset_index(drop=True)

y = df_input["PRICE"].reset_index(drop=True)

In [84]:
X.dtypes

PROPERTY TYPE            object
BEDS                    float64
BATHS                   float64
YEAR BUILT              float64
DAYS ON MARKET            int64
HOA/MONTH               float64
min_dist_to_grocery     float64
min_dist_to_mural       float64
min_dist_to_art         float64
count_grocery_0.5_mi      int64
count_murals_0.5_mi       int64
count_art_0.5_mi          int64
dtype: object

In [85]:
# make dummies for categorical variables
X_cat = X.select_dtypes(include=['object'])

one_hot_encoder = OneHotEncoder()
X_encoded = one_hot_encoder.fit_transform(X_cat)
X_encoded_labeled = pd.DataFrame(X_encoded.toarray(), columns=one_hot_encoder.categories_[0].tolist())

# combine with numerical data
X_num = X.select_dtypes(include=['int', 'float'])
X_input = pd.concat([X_num, X_encoded_labeled], axis=1)

In [86]:
X_input.head()

,BEDS,BATHS,YEAR BUILT,DAYS ON MARKET,HOA/MONTH,min_dist_to_grocery,min_dist_to_mural,min_dist_to_art,count_grocery_0.5_mi,count_murals_0.5_mi,count_art_0.5_mi,Condo/Co-op,Multi-Family (2-4 Unit),Single Family Residential,Townhouse
0,2.0,1.0,1908.0,5,338.0,0.459995,0.324022,0.675326,1,3,0,1.0,0.0,0.0,0.0
1,2.0,2.0,2008.0,5,335.0,0.718396,0.730189,0.573372,0,0,0,1.0,0.0,0.0,0.0
2,4.0,2.0,1915.0,5,0.0,0.447490,1.123216,2.167356,1,0,0,0.0,1.0,0.0,0.0
3,1.0,1.0,2001.0,5,360.0,0.223117,0.842772,0.449744,4,0,2,1.0,0.0,0.0,0.0
4,3.0,2.0,1905.0,5,0.0,0.444903,1.218160,2.617911,1,0,0,0.0,0.0,1.0,0.0


In [87]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_input, y, test_size=0.2)

# TODO: make validation set

In [93]:
# Create the linear regression model
lr = LinearRegression()

# Train the model on the training data
lr.fit(X_train, y_train)

# Make predictions on new data
# X_new = np.array([[3, 5]])
predictions = lr.predict(X_test)

preds = pd.Series(predictions, index=y_test.index)

In [96]:
df_preds = pd.concat([y_test, preds], axis=1)

In [97]:
df_preds

,PRICE,0
183,219999,107324.617857
108,399500,633328.526231
209,625000,321742.371722
15,368500,195742.393250
284,355000,210526.198623
...,...,...
122,410000,815097.728511
214,1129555,839206.093255
80,425000,263914.205547
282,284500,431774.929668


In [105]:
pd.DataFrame(lr.coef_, index=X_input.columns, columns=["coef"])

,coef
BEDS,15040.396914
BATHS,151536.455275
YEAR BUILT,2756.554070
DAYS ON MARKET,-714.397312
HOA/MONTH,128.973620
min_dist_to_grocery,-21604.119257
min_dist_to_mural,-25927.274606
min_dist_to_art,-24052.449211
count_grocery_0.5_mi,54641.780808
count_murals_0.5_mi,10298.821726
